# 3rd Level Model Structure: Everything (Linking Stages)

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

In [2]:
from pyomo import environ as pe
from modules.global_set import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

# Fixing Redundent Stream Variables

In [10]:
# condenser
model.condenser.VLE_block.n_ave.fix(58)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [11]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

In [12]:
check_DOF(pe,model)

Active Equality Constraints:	 1980
Active Inequality Constraints:	 0
Active Variables:		 2163
Fixed Variables:		 169
DOF:				 14


# Load Operating Parameters

In [13]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.setub(30+273.15)
# model.condenser.L['out'].fix(2)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(1500)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(58)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(0.3333)
    model.reactive[j].z['H2'].fix(0.6666)
    model.reactive[j].z['C30H62'].fix(0.0001)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    model.reactive[j].Q_main.fix(0)
    # model.reactive[j].T.setlb(270+273.15)

In [14]:
check_DOF(pe,model)

Active Equality Constraints:	 1980
Active Inequality Constraints:	 0
Active Variables:		 2163
Fixed Variables:		 181
DOF:				 2


In [15]:
model.obj = pe.Objective(expr = model.reactive[1].T ,sense=pe.maximize)

In [16]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [17]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [18]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:     6626
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

  75 -5.1273729e+02 5.46e-01 1.19e+05  -1.0 3.06e+03    -  1.23e-03 1.47e-03f  1
  76 -5.1275103e+02 5.45e-01 3.80e+05  -1.0 3.15e+03    -  4.75e-03 1.54e-03h  2
  77 -5.1275389e+02 5.45e-01 1.27e+06  -1.0 3.22e+03    -  7.72e-03 3.21e-04h  4
  78 -5.1275639e+02 5.45e-01 3.61e+06  -1.0 3.24e+03    -  9.92e-03 2.80e-04h  4
  79 -5.1275858e+02 5.45e-01 1.48e+07  -1.0 3.25e+03    -  1.18e-02 2.45e-04h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -5.1276050e+02 5.45e-01 1.17e+08  -1.0 3.27e+03    -  1.61e-02 2.15e-04h  4
  81 -5.1276218e+02 5.45e-01 6.87e+08  -1.0 3.28e+03    -  8.30e-03 1.88e-04h  4
  82 -5.1276364e+02 5.45e-01 9.07e+09  -1.0 3.29e+03    -  1.70e-02 1.64e-04h  4
  83 -5.1276621e+02 5.45e-01 6.93e+10  -1.0 3.30e+03    -  8.18e-03 2.88e-04h  3
  84 -5.1276815e+02 5.44e-01 1.48e+12  -1.0 3.32e+03    -  1.77e-02 2.16e-04h  3
  85 -5.1276960e+02 5.44e-01 1.92e+13  -1.0 3.33e+03    -  7.99e-03 1.62e-04h  3
  86 -5.1277396e+02 5.44e-01

 169 -5.3423258e+02 1.47e-03 6.37e+06  -1.7 8.90e+00    -  1.00e+00 2.40e-04h 13
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170 -5.3423361e+02 1.47e-03 6.52e+06  -1.7 8.87e+00    -  1.00e+00 2.40e-04h 13
 171 -5.3423464e+02 1.47e-03 6.67e+06  -1.7 8.85e+00    -  1.00e+00 2.40e-04h 13
 172 -5.3423568e+02 1.47e-03 6.82e+06  -1.7 8.82e+00    -  1.00e+00 2.41e-04h 13
 173 -5.3845079e+02 2.59e+00 4.07e+05  -1.7 8.80e+00    -  1.00e+00 9.86e-01w  1
 174 -5.3947894e+02 2.39e+00 3.59e+07  -1.7 4.76e+01    -  2.75e-01 4.48e-02w  1
 175 -5.3990491e+02 2.29e+00 6.80e+07  -1.7 2.47e+01    -  2.53e-02 4.23e-02w  1
 176 -5.3423670e+02 1.47e-03 6.98e+06  -1.7 3.32e+01    -  1.00e+00 2.41e-04h 12
 177 -5.3423773e+02 1.47e-03 7.14e+06  -1.7 8.78e+00    -  1.00e+00 2.41e-04h 13
 178 -5.3423876e+02 1.47e-03 7.30e+06  -1.7 8.76e+00    -  1.00e+00 2.41e-04h 13
 179 -5.3423978e+02 1.47e-03 7.45e+06  -1.7 8.75e+00    -  1.00e+00 2.41e-04h 13
iter    objective    inf_pr 

 261 -5.6749047e+02 1.90e-02 5.90e+07  -1.7 2.65e+00    -  1.00e+00 3.12e-02h  6
 262 -5.6682549e+02 1.22e+00 3.54e+05  -1.7 2.69e+00    -  1.00e+00 1.00e+00w  1
 263 -5.6721185e+02 2.12e-01 5.78e+05  -1.7 1.58e+00    -  1.00e+00 1.00e+00w  1
 264 -5.6721489e+02 3.47e-05 1.92e+16  -1.7 2.07e-01  17.0 1.00e+00 1.00e+00h  1
 265r-5.6721489e+02 3.47e-05 1.00e+03  -1.7 0.00e+00  19.2 0.00e+00 0.00e+00R  1
 266r-5.6721273e+02 5.17e-05 1.79e+04  -1.7 2.03e+01    -  1.03e-01 4.04e-03f  1
 267r-5.6720533e+02 2.59e-04 1.65e+04  -1.7 2.12e+01    -  2.40e-02 1.41e-02f  1
 268r-5.6718912e+02 1.31e-03 1.57e+04  -1.7 2.11e+01    -  3.95e-02 3.30e-02f  1
 269r-5.6701796e+02 1.65e-01 1.51e+04  -1.7 2.09e+01    -  2.68e-01 4.10e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270r-5.6700539e+02 4.26e-02 4.54e+03  -1.7 1.72e+01    -  5.91e-01 7.47e-01f  1
 271r-5.6700936e+02 8.60e-03 2.99e+03  -1.7 1.35e+01    -  6.27e-01 1.00e+00f  1
 272r-5.6701407e+02 5.57e-03

 353 -5.7314665e+02 8.40e-06 3.14e+03  -3.8 5.45e-06   8.7 4.19e-01 4.19e-01s 22
 354 -5.7314665e+02 3.86e-06 2.76e+03  -3.8 1.33e-05   8.2 3.06e-01 3.06e-01s 22
 355 -5.7314665e+02 2.25e-06 2.28e+03  -3.8 2.05e-05   7.7 2.64e-01 2.64e-01s 22
 356 -5.7314665e+02 1.04e-05 1.57e+03  -3.8 2.46e-05   7.2 3.61e-01 3.61e-01s 22
 357 -5.7314665e+02 1.48e-05 8.88e+02  -3.8 4.91e-05   6.7 4.49e-01 4.49e-01s 22
 358 -5.7314665e+02 1.38e-05 5.64e+02  -3.8 8.81e-05   6.3 3.65e-01 3.65e-01s 22
 359 -5.7314665e+02 1.20e-05 3.88e+02  -3.8 1.30e-04   5.8 3.11e-01 3.11e-01s 22
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 360 -5.7314665e+02 1.04e-05 2.93e+02  -3.8 2.10e-04   5.3 2.43e-01 2.43e-01s 22
 361 -5.7314665e+02 8.72e-06 9.66e+03  -5.7 3.77e-04   4.8 6.09e-01 1.82e-01h  1
 362 -5.7314665e+02 7.10e-06 9.39e+03  -5.7 7.39e-04   4.4 3.25e-01 2.13e-01h  1
 363 -5.7314665e+02 6.31e-06 6.15e+04  -5.7 1.46e-03   3.9 6.06e-01 1.27e-01h  1
 364 -5.7314665e+02 5.49e-06

 446 -5.7314667e+02 2.89e-07 2.14e+01  -5.7 4.99e-05   5.8 1.00e+00 1.00e+00H  1
 447 -5.7314667e+02 2.40e-07 1.52e+03  -5.7 2.11e-05   6.2 3.45e-01 4.77e-01h  2
 448 -5.7314667e+02 2.23e-07 4.14e+03  -5.7 6.36e-05   5.8 3.23e-01 1.00e+00H  1
 449 -5.7314667e+02 1.74e-07 5.30e+03  -5.7 1.94e-05   6.2 1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 450 -5.7314667e+02 1.88e-07 1.06e+03  -5.7 7.00e-05   5.7 7.39e-01 1.00e+00H  1
 451 -5.7314667e+02 1.89e-07 2.09e+01  -5.7 1.65e-05   6.1 1.00e+00 1.00e+00H  1
 452 -5.7314667e+02 3.41e-07 6.98e+02  -5.7 9.86e-05   5.7 6.08e-01 6.76e-01H  1
 453 -5.7314667e+02 2.99e-07 2.67e+03  -5.7 2.80e-05   6.1 7.46e-03 2.27e-01f  3
 454 -5.7314667e+02 1.83e-07 2.67e+03  -5.7 2.02e-04   5.6 1.28e-01 8.42e-02h  1
 455 -5.7314667e+02 2.45e-07 2.81e+03  -5.7 2.10e-05   6.0 1.99e-01 1.00e+00F  1
 456 -5.7314667e+02 1.79e-07 1.44e+03  -5.7 2.84e-04   5.6 5.90e-02 6.68e-02h  2
 457 -5.7314667e+02 1.75e-07

In [19]:
update_dual(pe,model)

In [20]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               29.90 	       -63.5675911994 	         0.1687014006 	         1.2809744469
              300.00 	         0.0000000000 	         1.7474112961 	         0.0002721140
----------------------------------------------------------------------------------------------------
Top
V	 0.16870140060505107
L	 0.044930848149232844
W	 0.25280460038681934
----------------------------------------------------------------------------------------------------
Bottom
L	 0.0002721140496310709
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Last Stage	Vapor		Liquid
H2 		8.622%		0.336%		 H2       	1.087%		0.116%
CO 		2.548%		0.183%		 CO       	0.385%		0.043%
CO2 		18.824%		11.437%		 CO2      	10.496%		2.459%
H2O 		1.333%		0.000%		 H2O      	14.596%		22.422%
C2H4 		11.341%		13.432%

In [21]:
model.reactive[1].kinetics_block.r_FT_total.value

0.2920169271697018

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/1_stage_condenser_n58_tmp.pickle','wb') as f:
#     pickle.dump(results,f)

### Check non-active constraints (dual = 0)

In [22]:
for i in model.dual:
    if abs(model.dual[i]) == 0:
        print('{:40s}'.format(str(i)),'\t\t',model.dual[i])

condenser.energy_block.dH_F_con[C30H62]  		 0.0
condenser.energy_block.dH_F_con[CO]      		 0.0
condenser.energy_block.dH_F_con[H2]      		 0.0
condenser.energy_block.H_F_con           		 0.0


# Iterative Data Analysis

In [23]:
import numpy as np
from matplotlib import pyplot as plt

In [24]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = 1 ,sense=pe.minimize)
model.condenser.L['out'].fixed = True
model.reactive[1].T.fixed = True
model.reactive[1].Q_main.fixed = False
# model.condenser.L['out'].setlb(6)
# for j in model.reactive:
#     model.reactive[j].Q_main.fixed=False
#     model.reactive[j].T.fixed=True

In [ ]:
check_DOF(pe,model)

In [ ]:
results = opt.solve(model,tee=True)

In [ ]:
update_dual(pe,model)

In [ ]:
Trange = np.linspace(300+273.15,200+273.15,50)

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['Re'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []

for i in model.reactive[1].r_total_comp:
    rf_data['r'][i] = []

for t in Trange:
    model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved, Solution T = {} K'.format(model.reactive[1].T.value))
    rf_data['T'].append(model.reactive[1].T.value)
    rf_data['Q'].append(model.reactive[1].Q_main.value)
    rf_data['V'].append(model.condenser.V['out'].value)
    rf_data['L'].append(model.condenser.L['P'].value)
    rf_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        rf_data['r'][i].append(model.reactive[1].r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.reactive[1].y['H2O'].value)
    rf_data['y_CO'].append(model.reactive[1].y['CO'].value)
    rf_data['y_H2'].append(model.reactive[1].y['H2'].value)
    rf_data['y_CO2'].append(model.reactive[1].y['CO2'].value)
    rf_data['r_WGS'].append(model.reactive[1].kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.reactive[1].kinetics_block.r_FT_total.value)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
line1 = ax1.plot(rf_data['T'],rf_data['Q'], 'b-')
line4 = ax1.plot(rf_data['T'],rf_data['Re'], 'k')
ax1.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax1.set_ylabel('Q (MW)', color='b',fontsize=18)
ax1.tick_params('y', colors='b',labelsize=18)
ax1.tick_params('x', colors='k',labelsize=18)
ax1.grid()

ax2 = ax1.twinx()
line2 = ax2.plot(rf_data['T'],rf_data['r_FT'], 'r')
line3 = ax2.plot(rf_data['T'],rf_data['r_WGS'], 'r--')
ax2.set_ylabel('$r_{FT}$: (kmol/s Carbon Number)', color='r',fontsize=18)
ax2.tick_params('y', colors='r',labelsize=18)

ax1.legend(line1+line2+line3+line4,['Q','$r_{FT}$','$r_{WGS}$','Reflux'],fontsize=18,loc=5)
ax1.set_title('Heat Removal Rate and Reaction Rate vs T',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['y_CO'])
ax.plot(rf_data['T'],rf_data['y_H2'])
ax.plot(rf_data['T'],rf_data['y_CO2'])
ax.plot(rf_data['T'],rf_data['y_H2O'])

ax.legend(['$y_{CO}$','$y_{H2}$','$y_{CO2}$','$y_{H2O}$'],fontsize=18)
ax.set_title('Gas Phase Inorganic Composition vs T',fontsize=18)

ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['r']['CO'])
ax.plot(rf_data['T'],rf_data['r']['H2'])
ax.plot(rf_data['T'],rf_data['r']['CO2'])
ax.plot(rf_data['T'],rf_data['r']['H2O'])

ax.legend(['$r_{CO}$','$r_{H2}$','$r_{CO2}$','$r_{H2O}$'],fontsize=18)
ax.set_title('Inorganic Reaction Rate vs T',fontsize=18)

ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
reaction_data = trans_product_mole(rf_data['r'])['unscaled']
reaction_data_scaled = trans_product_mole(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data_scaled['c1'], reaction_data_scaled['c2'],reaction_data_scaled['c3'],reaction_data_scaled['c4'],\
            reaction_data_scaled['napha'],reaction_data_scaled['gasoline'],reaction_data_scaled['diesel'],reaction_data_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mole) vs T',fontsize=18)
ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data['c1'], reaction_data['c2'],reaction_data['c3'],reaction_data['c4'],\
            reaction_data['napha'],reaction_data['gasoline'],reaction_data['diesel'],reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (Mole) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
mass_data = {}
for i in m.COMP_ORG:
    mass_data[i] = np.array(rf_data['r'][i])*u.cal_MW(i)

In [ ]:
mass_reaction_data = u.trans_product(mass_data)
mass_reaction_scaled = u.trans_product_scaled(mass_data)

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_scaled['c1'], mass_reaction_scaled['c2'],mass_reaction_scaled['c3'],mass_reaction_scaled['c4'],\
            mass_reaction_scaled['napha'],mass_reaction_scaled['gasoline'],mass_reaction_scaled['diesel'],mass_reaction_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mass) vs T',fontsize=18)
ax.set_ylabel('Mass Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_data['c1'], mass_reaction_data['c2'],mass_reaction_data['c3'],mass_reaction_data['c4'],\
            mass_reaction_data['napha'],mass_reaction_data['gasoline'],mass_reaction_data['diesel'],mass_reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (mass) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kg/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()